In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

In [2]:
df = pd.read_csv('score.csv', header=None)

# 鳳凰卓東風戦順位データ
- 0〜3→東家点数
- 4~7→最終点数
- 8→？？？(1しかない)
- 9~11→0視点との点差(万点)
- 12→ABCDさんの順位パターン全24通り(0~23)のうちどれだったか（actual）、最終的に予測するもの
- 13→東N局(0~3)

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,250,250,250,250,382,216,268,134,1,0.00,0.00,0.00,2,0
1,245,261,247,247,382,216,268,134,1,-0.16,-0.02,-0.02,2,1
2,235,310,247,208,382,216,268,134,1,-0.75,-0.12,0.27,2,1
3,325,220,247,208,382,216,268,134,1,1.05,0.78,1.17,2,2
4,325,220,272,183,382,216,268,134,1,1.05,0.53,1.42,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206607,200,440,120,240,220,420,120,240,1,-2.40,0.80,-0.40,13,3
206608,250,250,250,250,357,384,105,154,1,0.00,0.00,0.00,7,0
206609,250,340,160,250,357,384,105,154,1,-0.90,0.90,0.00,7,1
206610,265,325,145,255,357,384,105,154,1,-0.60,1.20,0.10,7,2


# 予測関数
- predict_proba→クラスの分類確率（合計1になる）

In [4]:
class Ranking:
    def __init__(self, clfs):
        self.clfs = clfs
        self.coeff = 10000

    def predict(self, kyoku, score):
        print('kyoku = ', kyoku)
        print('score = ', score)
        
        feature = np.array([[(score[0]-score[1])/self.coeff, (score[0]-score[2])/self.coeff, (score[0]-score[3])/self.coeff]])
        print('feature = ', feature)
        
        proba = self.clfs[kyoku].predict_proba(feature)[0]
        print('proba = ', proba)
        
        
        return np.array([ \
                         [ \
                          proba[0]+proba[1]+proba[2]+proba[3]+proba[4]+proba[5], \
                          proba[6]+proba[7]+proba[8]+proba[9]+proba[10]+proba[11], \
                          proba[12]+proba[13]+proba[14]+proba[15]+proba[16]+proba[17], \
                          proba[18]+proba[19]+proba[20]+proba[21]+proba[22]+proba[23], \
                         ], \
                         [ \
                          proba[6]+proba[7]+proba[12]+proba[13]+proba[18]+proba[19], \
                          proba[0]+proba[1]+proba[14]+proba[15]+proba[20]+proba[21], \
                          proba[2]+proba[3]+proba[8]+proba[9]+proba[22]+proba[23], \
                          proba[4]+proba[5]+proba[10]+proba[11]+proba[16]+proba[17], \
                         ], \
                         [ \
                          proba[8]+proba[10]+proba[14]+proba[16]+proba[20]+proba[22], \
                          proba[2]+proba[4]+proba[12]+proba[17]+proba[18]+proba[23], \
                          proba[0]+proba[5]+proba[6]+proba[11]+proba[19]+proba[21], \
                          proba[1]+proba[3]+proba[7]+proba[9]+proba[13]+proba[15], \
                         ], \
                         [ \
                          proba[9]+proba[11]+proba[15]+proba[17]+proba[21]+proba[23], \
                          proba[3]+proba[5]+proba[13]+proba[16]+proba[19]+proba[22], \
                          proba[1]+proba[4]+proba[7]+proba[10]+proba[18]+proba[20], \
                          proba[0]+proba[2]+proba[6]+proba[8]+proba[12]+proba[14], \
                         ] \
                        ])


In [5]:
if __name__ == '__main__':
    clfs = []
    
    for i in range(4):
        clfs.append(LogisticRegression(tol=1e-6, random_state=0, verbose=0))
        clf = clfs[-1]
        x = df[df[13] == i].iloc[:, 9:12]
        y = df[df[13] == i].iloc[:, 12]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
        clf.fit(x_train, y_train)

        print('{} Accuracy (train) :'.format(i), clf.score(x_train, y_train))
        print('{} Accuracy (test)  :'.format(i), clf.score(x_test, y_test))

    with open('score_predictor.bin', mode='wb') as f:
        pickle.dump(clfs, f)

,9,10,11
0,0.00,0.00,0.00
6,0.00,0.00,0.00
7,0.30,0.40,0.30
8,0.60,0.40,0.60
9,1.34,1.14,2.08


0    2
6    0
7    0
8    0
9    0
Name: 12, dtype: int64

0 Accuracy (train) : 0.07244196044711952
0 Accuracy (test)  : 0.07184502539025767


,9,10,11
1,-0.16,-0.02,-0.02
2,-0.75,-0.12,0.27
11,1.32,2.04,3.08
17,0.41,1.24,2.07
23,0.00,0.80,-0.80


1      2
2      2
11     0
17     9
23    23
Name: 12, dtype: int64

...

3      2
4      2
12     0
18     9
24    23
Name: 12, dtype: int64

2 Accuracy (train) : 0.23610527200874465
2 Accuracy (test)  : 0.2376405963902694


,9,10,11
5,1.47,0.95,2.26
13,1.83,2.47,3.46
19,0.41,1.14,0.73
20,0.41,1.72,0.15
27,-0.70,-1.69,-2.05


5      2
13     0
19     9
20     9
27    23
Name: 12, dtype: int64

3 Accuracy (train) : 0.3883307830483667
3 Accuracy (test)  : 0.3838869916175101


# 予測実行

In [6]:
# f = np.array([25000, 25000, 33000, 17000])
# f = np.array([50000, 25000, 25000, 0])
f = np.array([25000, 25000, 25000, 25000])
p = pickle.load(open('score_predictor.bin', 'rb'))
r = Ranking(p)
r.predict(1, f)

kyoku =  1
score =  [25000 25000 25000 25000]
feature =  [[0. 0. 0.]]
proba =  [0.03780821 0.03477596 0.03975546 0.03277106 0.04091074 0.03467876
 0.04390173 0.04205591 0.04580837 0.0474545  0.04279215 0.04817692
 0.04524193 0.03650204 0.04599467 0.04850306 0.04050735 0.04724344
 0.03701322 0.03571082 0.03845481 0.0462948  0.03701013 0.05063392]


array([[0.2207002 , 0.27018959, 0.2639925 , 0.24511771],
       [0.24042566, 0.25183152, 0.25343344, 0.25430937],
       [0.25056749, 0.26079872, 0.24657125, 0.24206254],
       [0.28830664, 0.21718017, 0.2360028 , 0.25851038]])